<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# download datasets
! wget 'https://raw.githubusercontent.com/wuyifan18/DeepLog/master/data/hdfs_train'
! wget 'https://raw.githubusercontent.com/wuyifan18/DeepLog/master/data/hdfs_test_normal'
! wget 'https://raw.githubusercontent.com/wuyifan18/DeepLog/master/data/hdfs_test_abnormal'

--2023-07-10 14:33:21--  https://raw.githubusercontent.com/wuyifan18/DeepLog/master/data/hdfs_train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 253021 (247K) [text/plain]
Saving to: ‘hdfs_train’

hdfs_train          100%[===================>] 247.09K  --.-KB/s    in 0.04s   

2023-07-10 14:33:21 (6.58 MB/s) - ‘hdfs_train’ saved [253021/253021]

--2023-07-10 14:33:21--  https://raw.githubusercontent.com/wuyifan18/DeepLog/master/data/hdfs_test_normal
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28730917 (27M) [text/plain]
Sav

In [3]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import argparse
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [25]:
# count rows of the hdfs_train

with open('/content/hdfs_train', "r") as file:
    totaln=0
    for line in file:
        totaln += 1

print('There are a total of {} lines'.format(totaln))

There are a total of 4855 lines


important point:
if you need to modify the elements of the line later on, using a list would be more suitable. If you want to ensure that the elements remain immutable, using a tuple would be appropriate.

In [35]:
window_size = 10
num_sessions = 0
inputs = []
outputs = []
with open('/content/hdfs_train', 'r') as f:
        for line in f:
            print(line)
            num_sessions += 1
            line = [int(n) - 1 for n in line.strip().split()]
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                outputs.append(line[i + window_size])
            break

print(line)
print(inputs)
print(outputs)
print(num_sessions)

5 5 5 22 11 9 11 9 11 9 26 26 26 23 23 23 21 21 21 

[4, 4, 4, 21, 10, 8, 10, 8, 10, 8, 25, 25, 25, 22, 22, 22, 20, 20, 20]
[[4, 4, 4, 21, 10, 8, 10, 8, 10, 8], [4, 4, 21, 10, 8, 10, 8, 10, 8, 25], [4, 21, 10, 8, 10, 8, 10, 8, 25, 25], [21, 10, 8, 10, 8, 10, 8, 25, 25, 25], [10, 8, 10, 8, 10, 8, 25, 25, 25, 22], [8, 10, 8, 10, 8, 25, 25, 25, 22, 22], [10, 8, 10, 8, 25, 25, 25, 22, 22, 22], [8, 10, 8, 25, 25, 25, 22, 22, 22, 20], [10, 8, 25, 25, 25, 22, 22, 22, 20, 20]]
[25, 25, 25, 22, 22, 22, 20, 20, 20]
1


In [37]:
# what happen if the line is smaller than windows size or equal

line = '5 5 5 22 11 9 11'
window_size = 10
num_sessions = 0
inputs = []
outputs = []

line = [int(n) - 1 for n in line.strip().split()]
for i in range(len(line) - window_size):
    inputs.append(line[i:i + window_size])
    outputs.append(line[i + window_size])

print(line)
print(inputs)
print(outputs)


[4, 4, 4, 21, 10, 8, 10]
[]
[]


In [38]:
line = '5 5 5 22 11 9 11 9 11 9'
window_size = 10
num_sessions = 0
inputs = []
outputs = []

line = [int(n) - 1 for n in line.strip().split()]
for i in range(len(line) - window_size):
    inputs.append(line[i:i + window_size])
    outputs.append(line[i + window_size])

print(line)
print(inputs)
print(outputs)

[4, 4, 4, 21, 10, 8, 10, 8, 10, 8]
[]
[]


In [40]:
import torch
from torch.utils.data import TensorDataset

name = 'hdfs_train'
window_size = 10
num_sessions = 0
inputs = []
outputs = []

with open('/content/' + name, 'r') as f:
        for line in f:
            num_sessions += 1
            line = [int(n) - 1 for n in line.strip().split()]
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                outputs.append(line[i + window_size])

print('Number of sessions({}): {}'.format(name, num_sessions))
print('Number of seqs({}): {}'.format(name, len(inputs)))
dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs))


Number of sessions(hdfs_train): 4855
Number of seqs(hdfs_train): 46575


In [41]:

inputs[0]

[4, 4, 4, 21, 10, 8, 10, 8, 10, 8]

In [42]:
outputs[0]

25

In [43]:
dataset.tensors